# AutoGluon Ensemble: GBM, LGBM, XGBoost, CatBoost, RF, Voting, Stacking

이 노트북은 AutoGluon을 사용하여 다양한 회귀 모델을 학습하고 앙상블(Voting, Stacking) 성능을 테스트합니다.
참조: `Plus_7_ensemble.ipynb`

In [ ]:
# AutoGluon 설치 (필요한 경우 주석 해제 후 실행)
# !pip install autogluon

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

np.set_printoptions(precision=4)

### 1. 데이터 로드 및 전처리

In [ ]:
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['MedHouseVal'] = data.target

print("데이터 크기:", df.shape)
df.head()

In [ ]:
# 학습/테스트 데이터 분리
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print("Train size:", train_data.shape)
print("Test size:", test_data.shape)

### 2. 모델 학습 설정 (Hyperparameters & Ensemble)

- **사용 모델**: GBM (LightGBM), XGB (XGBoost), CAT (CatBoost), RF (Random Forest)
- **앙상블 전략**:
    - **Voting (Weighted Ensemble)**: AutoGluon은 기본적으로 성능이 좋은 모델들의 가중 평균(Weighted Ensemble)을 생성합니다.
    - **Stacking**: `num_bag_folds`를 설정하여 K-Fold Bagging 및 Stacking을 수행합니다.

In [ ]:
label = 'MedHouseVal'
eval_metric = 'mean_squared_error'  # 회귀 문제 평가 지표

# 1. 특정 모델 지정 (GBM, XGB, CAT, RF)
# AutoGluon에서 'GBM'은 LightGBM을 의미합니다.
hyperparameters = {
    'GBM': {}, 
    'XGB': {}, 
    'CAT': {}, 
    'RF': {}
}

# 2. Stacking 설정
# num_bag_folds >= 2 이면 스태킹(Stacking)이 활성화됩니다.
num_bag_folds = 5  
num_stack_levels = 1 # 스태킹 레이어 수 (0이면 Bagging만, 1 이상이면 Stacking)

save_path = 'ag_models_california_housing'

### 3. 학습 실행

In [ ]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path).fit(
    train_data,
    hyperparameters=hyperparameters,
    num_bag_folds=num_bag_folds,
    num_stack_levels=num_stack_levels,
    fit_weighted_ensemble=True  # Weighted Ensemble (Voting 유사 효과) 활성화
)

### 4. 성능 평가 및 리더보드 확인

학습된 모델들의 성능을 비교합니다. 
- `WeightedEnsemble_L2` (또는 L3): 앙상블 모델 (Voting 역할)
- `LightGBM_BAG_L1` 등: Bagging/Stacking된 개별 모델들

In [ ]:
# 리더보드 출력 (테스트 데이터 기준 평가)
leaderboard = predictor.leaderboard(test_data, silent=False)

print("\n--- Top Models ---")
print(leaderboard.head())

### 5. 최종 평가

테스트 데이터 전체에 대한 최종 평가 점수(MSE)를 확인합니다.

In [ ]:
results = predictor.evaluate(test_data)
print("Final Test Results:", results)

### 6. 예측 결과 확인

In [ ]:
y_pred = predictor.predict(test_data)
print("First 5 predictions:\n", y_pred.head())
print("\nFirst 5 actual values:\n", test_data[label].head())